immanquablement des petites corrections a faire quand on traite les auteurs

# 1er episode : 26 janv 2025

et voir le contenu de `scripts/store_all_auteurs_from_episode.py`

In [ ]:
from mongo_episode import Episode
from mongo_auteur import Auteur, AuthorChecker

In [ ]:
import datetime

date = datetime.datetime(2025, 1, 26)

In [ ]:
import sys

episode = Episode.from_date(date=date)
if not episode:
    print(f"Episode du {date.strftime('%d/%m/%Y')} n'existe pas")
    # stoppe la cellule mais continue le notebook
    # raise RuntimeError("Arrêt de l'exécution de la cellule")
    # stoppe le notebook
    sys.exit(1)

print(episode)

auteurs = episode.get_all_auteurs()
print(auteurs)

ac = AuthorChecker(episode)

for auteur in auteurs:
    auteur_corrige = ac.check_author(auteur)
    if auteur_corrige is not None:
        print(f"{auteur} -> {auteur_corrige}")
        aut = Auteur(auteur_corrige)
        print(f"Est-ce que {aut.nom} existait en base ? {aut.exists()}")
        aut.keep()


        _oid: 679649e267b093aaae847524
        Date: 26 Jan 2025 10:59
        Titre: Les nouveaux romans de Leïla Slimani, Pierre Lemaître, Jeanne Rivière, Nathalie Azoulai, Milena Agus
        Description: durée : 00:47:56 - Le Masque et la Plume - par : Rebecca Manzoni - Une saga familiale à travers trois générations de femmes, entre le Maroc et la France ; une histoire d'amour et une réflexion sur la judéité ; un roman filial et d'espionnage dans la Guerre Froide ; amitié, désir, musique punk sans les années 90 ; littérature et amour en Sardaigne. - invités : Arnaud Viviant, Laurent CHALUMEAU, Patricia Martin, Elisabeth Philippe - Arnaud Viviant : Critique littéraire (Revue Regards), Laurent Chalumeau : Journaliste rock, scénariste, dialoguiste, romancier, Patricia Martin : Journaliste, critique littéraire et productrice chez France Inter, Elisabeth Philippe : Critique littéraire (L'Obs) - réalisé par : Guillaume Girault
        URL de téléchargement: https://rf.proxycast.org/f421

on va corriger Neige Sinno

In [ ]:
neige = Auteur("Neige Sinno")
neige.keep()